# Colab Starter Notebook — Practical Deep Learning
Created: 2025-09-03 09:34 UTC

This notebook is designed for engineering students to **start training deep learning models on Google Colab** with a GPU.

**What you'll do:**
1. Turn on GPU in Colab.
2. Check your environment.
3. Train a small image classifier (MNIST sample) using **fastai**.
4. Evaluate, export, and turn in your results.
5. Do a short assignment at the end.

> **Instructor note:** Duplicate this notebook per lesson. Keep the structure (Setup → Data → Train → Evaluate → Save → Assignment).


## 1) Enable GPU in Colab
- Go to **Runtime → Change runtime type → Hardware accelerator → GPU → Save**.
- Then run the cells below.


In [ ]:
# Quick GPU check (works only on Colab with GPU)
!nvidia-smi || echo "No NVIDIA GPU detected (that's OK if you're not on Colab)."


In [ ]:
# 2) Environment setup
# If running on Colab, install/upgrade fastai. On local machines you can comment this out.
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip -q install -U fastai fastdownload timm
else:
    print("Not in Colab (detected). If packages are missing, install fastai: pip install fastai fastdownload timm")


In [ ]:
# 3) Imports and device info
import torch
from fastai.vision.all import *
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU")


In [ ]:
# 4) Reproducibility (set seeds)
set_seed(42, reproducible=True)


## 2) Data
We'll use **MNIST_SAMPLE** (a tiny subset of MNIST: 3s and 7s) for a very fast demo.

> **Tip:** Switch to a bigger dataset later (e.g., `URLs.MNIST`, `URLs.PETS`, or a custom dataset).


In [ ]:
from fastai.data.external import untar_data, URLs
from fastai.vision.all import ImageDataLoaders

path = untar_data(URLs.MNIST_SAMPLE)  # very small dataset: digits 3 vs 7
print("Data path:", path)

dls = ImageDataLoaders.from_folder(
    path,
    train='train',
    valid='valid',
    valid_pct=None,     # use provided valid split
    item_tfms=Resize(224),  # adapt images to a size suitable for pretrained models
    bs=64
)
dls.show_batch(max_n=8)


## 3) Train a classifier
We'll train a **ResNet18** classifier with **fastai** and measure accuracy.


In [ ]:
learn = vision_learner(dls, resnet18, metrics=accuracy)
learn.fine_tune(1)  # keep it fast; increase epochs for better accuracy


## 4) Evaluate
Look at sample predictions and the confusion matrix.


In [ ]:
# Sample predictions
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(4,4))
plt.show()

# Show top losses
interp.plot_top_losses(6, nrows=2, figsize=(6,6))
plt.show()


## 5) Save and Inference
Export your trained model and try loading it back for inference.


In [ ]:
# Export the learner to a file
model_path = 'mnist_sample_resnet18.pkl'
learn.export(model_path)
print(f"Model exported to {model_path}")

# Example: load the exported model and predict on a single image
learn_inf = load_learner(model_path)
test_img = (path/'valid'/'3').ls()[0]  # pick one example
print("Testing on image:", test_img)
pred, pred_idx, probs = learn_inf.predict(test_img)
print("Prediction:", pred)
print("Probabilities:", probs)


## 6) (Optional) Timings & Resources
If you need to compare GPU vs CPU, measure timings.


In [ ]:
import time
start = time.time()
_ = learn.dls.valid.one_batch()
end = time.time()
print(f"One validation batch time: {end-start:.4f}s on this hardware.")


## 7) Assignment (Turn this in)
1. **Change the model or hyperparameters:** try `resnet34` or increase `fine_tune(3)` and report the new accuracy.
2. **Augmentations:** add `batch_tfms=aug_transforms(do_flip=False, max_rotate=15, max_zoom=1.1)` to `ImageDataLoaders.from_folder` and report the impact.
3. **Learning rate finder:** run `learn.lr_find()` and choose a learning rate; re-train and compare.
4. **Explain:** In 3–5 sentences, explain the difference between **loss** and **metric** in your own words.
5. **Screenshot:** Include a screenshot of your confusion matrix and top losses.

> **Deliverable:** A short report (Markdown cell below) plus the notebook with your final model exported.


### Your short report
*(Write here: what you changed, your observations, final accuracy, and a brief explanation of loss vs metric.)*


## 8) Teaching Notes (Instructor Only)
- Emphasize **GPU vs CPU** and session limits in Colab.
- Highlight the training loop high-level flow: **data → model → loss → optimizer → metric**.
- Encourage students to **iterate quickly** on small datasets; scale up later.
- Common pitfalls: not enabling GPU; RAM/session resets; forgetting to export models.


## 9) Troubleshooting
- **No GPU?** Re-check *Runtime → Change runtime type → GPU* and re-run setup.
- **ImportError (fastai not found)?** Re-run the install cell.
- **Out of memory (OOM)?** Lower `bs` (batch size), e.g., `bs=32`.
- **Slow training?** Reduce image size in `Resize(128)` or use a smaller model.
- **Session reset?** Save your work to Google Drive or GitHub frequently.


## Appendix: (Optional) Minimal PyTorch Training Loop
For students curious about what's under the hood, here's a very small PyTorch loop on random data.


In [ ]:
# Educational only — a tiny loop on fake data to show the pattern
import torch
from torch import nn, optim

X = torch.randn(512, 28*28)
y = (X.sum(dim=1) > 0).long()  # fake labels

model = nn.Sequential(nn.Linear(28*28, 128), nn.ReLU(), nn.Linear(128, 2))
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(3):
    model.train()
    opt.zero_grad()
    preds = model(X)
    loss = loss_fn(preds, y)
    loss.backward()
    opt.step()
    with torch.no_grad():
        acc = (preds.argmax(dim=1) == y).float().mean().item()
    print(f"Epoch {epoch+1}: loss={loss.item():.4f}, acc={acc:.3f}")
